# Решение #2 - Равное голосование

Решение использует следующие алгоритмы:
* Косинусное расстояние
* Расстояние Дамерау-Левенштейна
* L2-расстояние
* N-граммы + метрика Индекс Тверского
* Расстояние Хэмминга
* Проверка наборов местоимений

Решение основано на объединении результатов разных алгоритмов.

## Загрузка данных

In [1]:
from src import *
base_data = load_dataset.load('../data/sample.json')

In [2]:
base_data

['Ты нашёл их или нет?',
 'Почему она так со мной поступает?',
 'Никто туда больше не ходит.',
 'У него с собой не было тогда денег.',
 'Почему они с нами так поступают?',
 'Он всю ночь стонал от сильной боли.',
 'Я больше не хочу с тобой играть.',
 'Тому было тогда всего тринадцать лет.',
 'Что сделал Том с деньгами?',
 'Том меня сейчас хочет видеть?',
 'Он даже меня не замечает.',
 'Тебе это всё нравится?',
 'У него тогда не было с собой денег.',
 'Я его больше не увижу.',
 'Почему она так с ней поступает?',
 'Я хотел бы учиться в Бостоне.',
 'Том и Мэри объявили сегодня о своей пбмолвке.',
 'Том этим сейчас занимается.',
 'Я не могу больше ждать.',
 'Мост очень длинный и высокий.',
 'Том был просто не готов.',
 'Пусть свиньи это едят.',
 'Я ничего не хочу делать.',
 'Ты хотел мне рассказать о свободе?',
 'Что пел Джон на сцене?',
 'Я написал влера письмо.',
 'Тому это тоже не нравится.',
 'Сколько сейчас времени в Париже?',
 'Мы не были готовы.',
 'Они их только что нашли.',
 'Тому 

In [3]:
data = [load_dataset.preprocess_text(text) for text in base_data]

In [4]:
data

['ты нашел их или нет',
 'почему она так со мной поступает',
 'никто туда больше не ходит',
 'у него с собой не было тогда денег',
 'почему они с нами так поступают',
 'он всю ночь стонал от сильной боли',
 'я больше не хочу с тобой играть',
 'тому было тогда всего тринадцать лет',
 'что сделал том с деньгами',
 'том меня сейчас хочет видеть',
 'он даже меня не замечает',
 'тебе это все нравится',
 'у него тогда не было с собой денег',
 'я его больше не увижу',
 'почему она так с ней поступает',
 'я хотел бы учиться в бостоне',
 'том и мэри объявили сегодня о своей пбмолвке',
 'том этим сейчас занимается',
 'я не могу больше ждать',
 'мост очень длинный и высокий',
 'том был просто не готов',
 'пусть свиньи это едят',
 'я ничего не хочу делать',
 'ты хотел мне рассказать о свободе',
 'что пел джон на сцене',
 'я написал влера письмо',
 'тому это тоже не нравится',
 'сколько сейчас времени в париже',
 'мы не были готовы',
 'они их только что нашли',
 'тому было больше не к кому обратить

## Группировка предложений

In [5]:
from tqdm.auto import tqdm
from ipywidgets import FloatProgress

# Функция группировки строк
def find_groups(data, base_data):
    groups = []                        # Cписок для сохранения групп
    threshold = 0.9                    # Ограничение для алгоритмов верификации рерайтов
    used = [False for _ in range(len(data))]    # Список меток использования строк
    for current_id in tqdm(range(len(data))):
        if (used[current_id]):
            continue
        current_group = []             # Текущая группа
        current_group.append(base_data[current_id])
        used[current_id] = True
        for next_id in range(current_id + 1, len(data)):
            if (used[next_id]):
                continue
            line1 = data[current_id]
            line2 = data[next_id]
            # Проверка строк с помощью алгоритмов верификации
            if (cosine_count.is_rewrite_cosine_count(line1, line2, threshold) or
               dam_lev_dist.is_rewrite_damerau_levenshtein_distance(line1, line2, threshold) or
               distance_L2.is_rewrite_distance_L2(line1, line2, threshold) or
               ngrams.is_rewrite_tverskiy_ngram(line1, line2, threshold)):
                if (pronouns.check_pronoun_correspondence(line1, line2) or(hamming_distance_custom.clean_hamming_distance(line1, line2) == 1)):
                    current_group.append(base_data[next_id])  # Добавление строки в текущую группу
                    used[next_id] = True
        groups.append(current_group)             # Добавление сформированной группы к результирующему списку
    return groups
   
        

In [6]:
groups = find_groups(data, base_data)

  0%|          | 0/412 [00:00<?, ?it/s]

## Результаты работы

### Количество найденных групп рерайтов

In [7]:
len(groups)

281

### Найденные группы

In [8]:
groups

[['Ты нашёл их или нет?'],
 ['Почему она так со мной поступает?'],
 ['Никто туда больше не ходит.', 'Никто больше туда не ходит.'],
 ['У него с собой не было тогда денег.',
  'У него тогда не было с собой денег.'],
 ['Почему они с нами так поступают?'],
 ['Он всю ночь стонал от сильной боли.'],
 ['Я больше не хочу с тобой играть.', 'Я не хочу с тобой больше играть.'],
 ['Тому было тогда всего тринадцать лет.'],
 ['Что сделал Том с деньгами?', 'Что сделёл Том с деньгами?'],
 ['Том меня сейчас хочет видеть?', 'Том хочет меня сейчас видеть?'],
 ['Он даже меня не замечает.'],
 ['Тебе это всё нравится?'],
 ['Я его больше не увижу.', 'Я больше его не увижу.'],
 ['Почему она так с ней поступает?'],
 ['Я хотел бы учиться в Бостоне.'],
 ['Том и Мэри объявили сегодня о своей пбмолвке.',
  'Том и Мэри объявили сегодня о своей помолвке.'],
 ['Том этим сейчас занимается.'],
 ['Я не могу больше ждать.', 'Я больше не могу ждать.'],
 ['Мост очень длинный и высокий.', 'Мост очень длинный и очень высоки

### Группы размера 1

In [9]:
groups1 = [group for group in groups if len(group) == 1]

In [10]:
len(groups1)

151

In [11]:
groups1

[['Ты нашёл их или нет?'],
 ['Почему она так со мной поступает?'],
 ['Почему они с нами так поступают?'],
 ['Он всю ночь стонал от сильной боли.'],
 ['Тому было тогда всего тринадцать лет.'],
 ['Он даже меня не замечает.'],
 ['Тебе это всё нравится?'],
 ['Почему она так с ней поступает?'],
 ['Я хотел бы учиться в Бостоне.'],
 ['Том этим сейчас занимается.'],
 ['Том был просто не готов.'],
 ['Я ничего не хочу делать.'],
 ['Тому это тоже не нравится.'],
 ['Сколько сейчас времени в Париже?'],
 ['Тому было больше не к кому обратиться.'],
 ['Он сказал, что вчера был дома.'],
 ['Они никогда меня не слушают.'],
 ['Я поймал сегодня три рыбы.'],
 ['Я хочу что-нибудь сделать для Тома.'],
 ['Я сделал всё правильно?'],
 ['Я тоже завтра пойду в университет.'],
 ['Мне никогда не нужно было столько денег.'],
 ['Том не может жить без Мэри.'],
 ['Почему они так с ним поступают?'],
 ['Что могло с ними случиться?'],
 ['Мы будем делать всё по-своему.'],
 ['Где был Том весь день?'],
 ['Я здесь часто ем.'],

### Группы размера 2

In [12]:
groups2 = [group for group in groups if len(group) == 2]

In [13]:
len(groups2)

129

In [14]:
groups2

[['Никто туда больше не ходит.', 'Никто больше туда не ходит.'],
 ['У него с собой не было тогда денег.',
  'У него тогда не было с собой денег.'],
 ['Я больше не хочу с тобой играть.', 'Я не хочу с тобой больше играть.'],
 ['Что сделал Том с деньгами?', 'Что сделёл Том с деньгами?'],
 ['Том меня сейчас хочет видеть?', 'Том хочет меня сейчас видеть?'],
 ['Я его больше не увижу.', 'Я больше его не увижу.'],
 ['Том и Мэри объявили сегодня о своей пбмолвке.',
  'Том и Мэри объявили сегодня о своей помолвке.'],
 ['Я не могу больше ждать.', 'Я больше не могу ждать.'],
 ['Мост очень длинный и высокий.', 'Мост очень длинный и очень высокий.'],
 ['Пусть свиньи это едят.', 'Пусть это свиньи едят.'],
 ['Ты хотел мне рассказать о свободе?', 'Ты хотел рассказать мне о свободе?'],
 ['Что пел Джон на сцене?', 'Что Джон пел на сцене?'],
 ['Я написал влера письмо.', 'Я написал вчера письмо.'],
 ['Мы не были готовы.', 'Мы были не готовы.'],
 ['Они их только что нашли.', 'Они только что их нашли.'],
 ['

### Группы размера 3

In [15]:
groups3 = [group for group in groups if len(group) == 3]

In [16]:
len(groups3)

1

In [17]:
groups3

[['Том не хочет никого видеть.',
  'Том не хочет сегодня никого видеть.',
  'Том сегодня не хочет никого видеть.']]

## Сохранение результатов

In [18]:
save_groups.save_groups(groups, "../output/solution2-result.json")